In [1]:
import pandas as pd

from lsh_utils import load_phoible
from lsh_utils import compare

from simphon import tokens

We can look at each token in our toy data set along with its phonemic transcription as a sequence of phonemes (between "/"):



In [2]:
# a toy dataset to work with
print(*tokens, sep='\n')

(ell) 〈Σοφία〉 /s o f i ɐ/
(ell) 〈Αλέξανδρος〉 /ɐ l ɛ k s ɐ n ð ɾ o s/
(eng) 〈Adam〉 /æ d ə m/
(eng) 〈Alex〉 /æ l ɪ k s/
(eng) 〈Alexander〉 /æ l ɪ k z æ n d ɚ/
(eng) 〈Alexis〉 /ə l ɛ k s ɪ s/
(eng) 〈Amit〉 /ɑ m i t/
(eng) 〈Andrew〉 /æ n d ɹ u/
(eng) 〈Brad〉 /b ɹ æ d/
(eng) 〈Bradley〉 /b ɹ æ d l i/
(eng) 〈Brett〉 /b ɹ ɛ t/
(eng) 〈Carl〉 /k ɑ ɹ l/
(eng) 〈Carlos〉 /k ɑ ɹ l oʊ s/
(eng) 〈Catherine〉 /k æ θ ə ɹ ɪ n/
(eng) 〈Charles〉 /t̠ʃ ɑ ɹ l z/
(eng) 〈Drew〉 /d ɹ u/
(eng) 〈Jennifer〉 /d̠ʒ ɛ n ə f ɚ/
(eng) 〈Jenny〉 /d̠ʒ ɛ n i/
(eng) 〈John〉 /d̠ʒ ɑ n/
(eng) 〈Johnny〉 /d̠ʒ ɑ n i/
(eng) 〈Jonathan〉 /d̠ʒ ɑ n ə θ ə n/
(eng) 〈Kat〉 /k æ t/
(eng) 〈Kathy〉 /k æ θ i/
(eng) 〈Katsuya〉 /k æ t s uː j ə/
(eng) 〈Matt〉 /m æ t/
(eng) 〈Matthew〉 /m æ θ j u/
(eng) 〈Michael〉 /m a ɪ k ə l/
(eng) 〈Mike〉 /m aɪ k/
(eng) 〈Nate〉 /n eɪ t/
(eng) 〈Nathan〉 /n eɪ θ ə n/
(eng) 〈Nathaniel〉 /n ə θ æ n j ə l/
(eng) 〈Nichole〉 /n ɪ k oʊ l/
(eng) 〈Nick〉 /n ɪ k/
(eng) 〈Phil〉 /f ɪ l/
(eng) 〈Philip〉 /f ɪ l ɪ p/
(eng) 〈Shinzo〉 /ʃ ɪ n z oʊ/
(eng) 〈Smith〉 /

We can see the PHOIBLE feature values as a 2-D matrix representation of each token as follows:

In [3]:
from IPython.display import display
pd.options.display.max_columns = None

for token in tokens:
    display(token.as_feature_matrix())

,s,o,f,i,ɐ
Phoneme,s,o,f,i,ɐ
Allophones,s,o,f,i̥ ĩ i,ɐ
Marginal,nan,False,nan,nan,False
SegmentClass,consonant,vowel,consonant,vowel,vowel
tone,0,0,0,0,0
stress,-,-,-,-,-
syllabic,-,+,-,+,+
short,-,-,-,-,-
long,-,-,-,-,-
consonantal,+,-,+,-,-


,ɐ,l,ɛ,k,s,ɐ,n,ð,ɾ,o,s
Phoneme,ɐ,l,ɛ,k,s,ɐ,n,ð,ɾ,o,s
Allophones,ɐ,lʲ l,ɛ̃ ɛ ɛ̥,k kʲ,s,ɐ,nʲ ŋ n,ð,ɾ,o,s
Marginal,False,nan,nan,nan,nan,False,nan,nan,nan,False,nan
SegmentClass,vowel,consonant,vowel,consonant,consonant,vowel,consonant,consonant,consonant,vowel,consonant
tone,0,0,0,0,0,0,0,0,0,0,0
stress,-,-,-,-,-,-,-,-,-,-,-
syllabic,+,-,+,-,-,+,-,-,-,+,-
short,-,-,-,-,-,-,-,-,-,-,-
long,-,-,-,-,-,-,-,-,-,-,-
consonantal,-,+,-,+,+,-,+,+,+,-,+


,æ,d,ə,m
Phoneme,æ,d,ə,m
Allophones,æ æː,t͉ dˡ d̪ dⁿ ɖ d dz d̚ d,ɜ ə ə ə,ɱ m̩ m
Marginal,nan,nan,nan,nan
SegmentClass,vowel,consonant,vowel,consonant
tone,0,0,0,0
stress,-,-,-,-
syllabic,+,-,+,-
short,-,-,-,-
long,-,-,-,-
consonantal,-,+,-,+


,æ,l,ɪ,k,s
Phoneme,æ,l,ɪ,k,s
Allophones,æ æː,ɭ l̩ˠ lˠ l̥ l l̪ˠ,ɪ,k ʃ ʒ,s
Marginal,nan,nan,nan,False,nan
SegmentClass,vowel,consonant,vowel,consonant,consonant
tone,0,0,0,0,0
stress,-,-,-,-,-
syllabic,+,-,+,-,-
short,-,-,-,-,-
long,-,-,-,-,-
consonantal,-,+,-,+,+


,æ,l,ɪ,k,z,æ,n,d,ɚ
Phoneme,æ,l,ɪ,k,z,æ,n,d,ɚ
Allophones,æ æː,ɭ l̩ˠ lˠ l̥ l l̪ˠ,ɪ,k ʃ ʒ,z z s͉,æ æː,n̩ n̪ ɳ n,t͉ dˡ d̪ dⁿ ɖ d dz d̚ d,ɚ
Marginal,nan,nan,nan,False,nan,nan,nan,nan,False
SegmentClass,vowel,consonant,vowel,consonant,consonant,vowel,consonant,consonant,vowel
tone,0,0,0,0,0,0,0,0,0
stress,-,-,-,-,-,-,-,-,-
syllabic,+,-,+,-,-,+,-,-,+
short,-,-,-,-,-,-,-,-,-
long,-,-,-,-,-,-,-,-,-
consonantal,-,+,-,+,+,-,+,+,-


...

,m,a,s,a,j,o,s,i
Phoneme,m,a,s,a,j,o,s,i
Allophones,mʲ m,ã ḁ a,s,ã ḁ a,j j̥,o,s,i̥ ĩ i
Marginal,nan,nan,nan,nan,nan,False,nan,nan
SegmentClass,consonant,vowel,consonant,vowel,consonant,vowel,consonant,vowel
tone,0,0,0,0,0,0,0,0
stress,-,-,-,-,-,-,-,-
syllabic,-,+,-,+,-,+,-,+
short,-,-,-,-,-,-,-,-
long,-,-,-,-,-,-,-,-
consonantal,+,-,+,-,-,-,+,-


,t,a,d,a,j,o,s,i
Phoneme,t,a,d,a,j,o,s,i
Allophones,t ts,ã ḁ a,d,ã ḁ a,j j̥,o,s,i̥ ĩ i
Marginal,nan,nan,nan,nan,nan,False,nan,nan
SegmentClass,consonant,vowel,consonant,vowel,consonant,vowel,consonant,vowel
tone,0,0,0,0,0,0,0,0
stress,-,-,-,-,-,-,-,-
syllabic,-,+,-,+,-,+,-,+
short,-,-,-,-,-,-,-,-
long,-,-,-,-,-,-,-,-
consonantal,+,-,+,-,-,-,+,-


,k,a,tsː,ɯ,d̠,a
Phoneme,k,a,tsː,ɯ,d̠,a
Allophones,k̟ʲ k,ã ḁ a,tsː,ɯ̥ ɯ ɯ,d̠ d̠͓ ɾ l̠ d̠ʲ,ã ḁ a
Marginal,nan,nan,nan,nan,False,nan
SegmentClass,consonant,vowel,consonant,vowel,consonant,vowel
tone,0,0,0,0,0,0
stress,-,-,-,-,-,-
syllabic,-,+,-,+,-,+
short,-,-,-,-,-,-
long,-,-,+,-,-,-
consonantal,+,-,+,-,+,-


,k,a,tsː,ɯ,d̠,a
Phoneme,k,a,tsː,ɯ,d̠,a
Allophones,k̟ʲ k,ã ḁ a,tsː,ɯ̥ ɯ ɯ,d̠ d̠͓ ɾ l̠ d̠ʲ,ã ḁ a
Marginal,nan,nan,nan,nan,False,nan
SegmentClass,consonant,vowel,consonant,vowel,consonant,vowel
tone,0,0,0,0,0,0
stress,-,-,-,-,-,-
syllabic,-,+,-,+,-,+
short,-,-,-,-,-,-
long,-,-,+,-,-,-
consonantal,+,-,+,-,+,-


,a,lʲ,e,k,sʲ,e,j
Phoneme,a,lʲ,e,k,sʲ,e,j
Allophones,nan,l̥ʲ lʲ,nan,k kʷ,sʲ,nan,j
Marginal,False,nan,False,nan,nan,False,nan
SegmentClass,vowel,consonant,vowel,consonant,consonant,vowel,consonant
tone,0,0,0,0,0,0,0
stress,-,-,-,-,-,-,-
syllabic,+,-,+,-,-,+,-
short,-,-,-,-,-,-,-
long,-,-,-,-,-,-,-
consonantal,-,+,-,+,+,-,-


Finally, we can compare and rank pairs of tokens based on their phonemic similarity:

In [4]:
pd.options.display.max_rows = None
compare(tokens)

ranking pairs by bitwise similarity: 0pair [00:00, ?pair/s]

,a,b,simhash difference (in bits),similarity score
0,(jpn) 〈かつら〉 /k a tsː ɯ d̠ a/,(jpn) 〈カツラ〉 /k a tsː ɯ d̠ a/,0,1.0
1,(eng) 〈Zach〉 /z æ k/,(eng) 〈Zack〉 /z æ k/,0,1.0
2,(heb) 〈צוֹפִית〉 /ts o f i t/,(heb) 〈צוּפִית〉 /ts u f i t/,470,0.847
3,(eng) 〈Amit〉 /ɑ m i t/,(heb) 〈עמת〉 /a m ɪ t/,560,0.818
4,(jpn) 〈ただよし〉 /t a d a j o s i/,(jpn) 〈まさよし〉 /m a s a j o s i/,571,0.814
5,(eng) 〈Zack〉 /z æ k/,(heb) 〈זך〉 /z a k/,586,0.809
6,(eng) 〈Zach〉 /z æ k/,(heb) 〈זך〉 /z a k/,586,0.809
7,(eng) 〈Zachariah〉 /z æ k ə ɹ aɪ ə/,(eng) 〈Zachary〉 /z æ k ə ɹ i/,598,0.805
8,(heb) 〈עמת〉 /a m ɪ t/,(hin) 〈अमित〉 /aː m ɪ t̪/,602,0.804
9,(eng) 〈Tsofit〉 /s oʊ f i t/,(heb) 〈צוּפִית〉 /ts u f i t/,614,0.8
